## LHC papers
Resources:
* https://github.com/kamyu104/LeetCode-Solutions/blob/master/Python/h-index.py
* https://www.analyticsvidhya.com/blog/2017/09/building-machine-learning-model-fun-using-orange/
* https://orange.biolab.si/blog/2016/07/18/network-analysis-with-orange/

In [6]:
import pandas as pd
import time
import random

## Functions

In [2]:
def normalize(df,nd={'title':'title_TeX',
                     'doi':'DOI'}):
    for k in nd.keys():
        if k in df:
            df[nd[k]]=df[k].apply(
                    lambda d: d[0] if isinstance(d,list) else d
                )
            #special cases
            if k=='title':
                df[nd[k]]=df[nd[k]].apply(
                    lambda d: d.get(k) if isinstance(d,dict) else d)
    return df

In [3]:
lhc_query='cn cms or cn atlas or cn lhcb or cn alice'

## Get query results

In [4]:
q='{} and collection:published'.format(lhc_query)
q=q.replace(' ','+')
#df=pd.read_json(
#    'https://inspirehep.net/search?p=find+{}&rg=250&of=recjson'.format(q))

In [9]:
def get_query(q):
    cc=pd.DataFrame()
    jrec=cc.shape[0]
    while jrec%250==0:
        query='http://inspirehep.net/search?ln=en&p={}&rg=250&jrec={}&of=recjson'.format(
              q,jrec+1)
        try:
            c=pd.read_json(query)
        except:
            c=pd.DataFrame()
        c=normalize(c)
        if not c.empty:
            cc=(cc.append(c)).reset_index(drop=True)
            jrec=cc.shape[0]
        else:
            jrec=jrec+1 # To exit the loop
        print('jrec',jrec)
        time.sleep(1)
    return cc

def get_citations(recid):
    cc=pd.DataFrame()
    jrec=cc.shape[0]
    while jrec%250==0:
        qcites='http://inspirehep.net/search?ln=en&p=refersto:recid:{}&rg=250&jrec={}&of=recjson'.format(
              recid,jrec+1)
        try:
            c=pd.read_json(qcites)
        except:
            c=pd.DataFrame()
        c=normalize(c)
        for k in ['recid','authors','DOI','publication_info']:
            if k not in c:
                c[k]=None

        if not c.empty:
            c['cites']=c.apply(lambda row: 
                               {'recid':row['recid'],
                                'authors':row['authors'],
                                'publication_info':row['publication_info'],
                                'DOI'    :row['DOI']
                               },   axis=1)
        if not c.empty:
            cc=(cc.append(c)).reset_index(drop=True)
            jrec=cc.shape[0]
        else:
            jrec=jrec+1 # To exit the loop
        print('jrec',jrec)
        time.sleep(1)
    return cc
def get_citations_list(recid):
    c=get_citations(recid)
    print(recid)
    time.sleep(random.randint(1,5))
    if not c.empty:
        return list(c['cites'].values)
    else:
        return None

In [10]:
lhc=get_query(q)

jrec 250


/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


jrec 500


/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


jrec 750


...

jrec 3250


/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


jrec 3500


/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


jrec 3584


In [ ]:
lhc.to_json('lhc.json.gz',compression='gzip')

In [11]:
df=lhc.copy()

## Add citations metadata

Add citations column

In [ ]:
s=time.time()
df['cites']=df['recid'].apply(get_citations_list)
tot=time.time()-s
print(tot)

jrec 1
1764794
jrec 3
1757579
jrec 1
1755250
jrec 2
1754075
jrec 5
1753976
jrec 6
1753680
jrec 2
1753375
jrec 5
1753215
jrec 6
1751230
jrec 2
1750601
jrec 12
1750186
jrec 7
1749379
jrec 2
1748735
jrec 1
1747892
jrec 5
1744608
jrec 3
1744603
jrec 6
1744267
jrec 1
1743890
jrec 3
1743799
jrec 9
1742786
jrec 4
1742776
jrec 2
1742608
jrec 1
1742099
jrec 12
1741996
jrec 4
1741056
jrec 7
1740108
jrec 11
1738833
jrec 9
1737508
jrec 5
1736896
jrec 3
1736895
jrec 5
1736729
jrec 11
1735737
jrec 3
1735729
jrec 4
1734235
jrec 7
1725476
jrec 8
1724439
jrec 29
1718338
jrec 1
1717867
jrec 7
1709434
jrec 15
1709180
jrec 18
1708907
jrec 28
1704316
jrec 87
1694380
jrec 9
1693823
jrec 1
1692605
jrec 15
1686334
jrec 9
1682904
jrec 9
1679809
jrec 16
1671831
jrec 16
1663960
jrec 1
1655573
jrec 14
1654830
jrec 9
1654572
jrec 3
1651511
jrec 8
1627633
jrec 5
1620051
jrec 10
1615477
jrec 1
1609450
jrec 32
1608166
jrec 9
1607572
jrec 34
1605750
jrec 17
1605749
jrec 250
jrec 251
1605397
jrec 40
1604886
jrec 1
1603

In [90]:
kk=df.copy()
kk['nc']=kk['cites'].apply(lambda l: len(l) if isinstance(l,list) else 0)
dfy=kk[kk.apply(lambda row: row['number_of_citations']==row['nc'],axis=1)].reset_index(drop=True)
dfn=kk[kk.apply(lambda row: row['number_of_citations']!=row['nc'],axis=1)].reset_index(drop=True)
dfn.shape[0],dfn.shape[0]+dfy.shape[0],df.shape[0]

(0, 195, 195)

In [69]:
dfn['cites']=dfn['recid'].apply(get_citations_list)

jrec 250


/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


jrec 500


/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


jrec 677
1605397
jrec 44
1603635
jrec 250


/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


jrec 412
1267508
jrec 114
1265220


In [71]:
df=(dfy.append(dfn)).reset_index(drop=True)

## Filter published papers (query already filtered!)

In [72]:
dfp=df[~df['publication_info'].isna()].reset_index(drop=True)

Get the Journal info from crossref api

In [73]:
journal={}

In [74]:
import time
def get_journal_info(row,journal=journal,abrv_journal='publication_info'
                                        ,abrv_journal_title='title'
                                        ,doi='doi'
                                        ,container_title='container_title'
                                        ,issn='ISSN'):
    
    try:
        j=row.get(abrv_journal).get(abrv_journal_title)
    except:
        j=None
    if j and not journal.get(j):
        journal.update( {j:{}} )
    
    if not journal.get(j) and row.get(doi):
        strdoi=row.get(doi)
        try:
            di=pd.read_json( 'https://api.crossref.org/works/{}'.format( strdoi  ) )
        except:
            di=pd.DataFrame()
        if j in journal and not journal.get(j).get(container_title):
            try:
                ct=di.loc['container-title'][0][0]
                journal[j].update({container_title:ct})

            except:
                pass
        if j in journal and not journal.get(j).get(issn):
            try:
                gis=di.loc['ISSN'].get('message')[0]
                journal[j].update({issn:gis})
            except:
                pass
        #Get a new journal crossref metadata here

        time.sleep(1)
        return di

In [75]:
di=dfp.apply(get_journal_info,axis=1)

In [76]:
journal

{'Eur.Phys.J.': {'ISSN': '1434-6044',
  'container_title': 'The European Physical Journal C'},
 'Europhys.Lett.': {'ISSN': '0295-5075',
  'container_title': 'Europhysics Letters (EPL)'},
 'Int.J.Mod.Phys.': {'ISSN': '0217-751X',
  'container_title': 'International Journal of Modern Physics A'},
 'J.Phys.': {'ISSN': '0954-3899',
  'container_title': 'Journal of Physics G: Nuclear and Particle Physics'},
 'JCAP': {'ISSN': '1475-7516',
  'container_title': 'Journal of Cosmology and Astroparticle Physics'},
 'JHEP': {'ISSN': '1029-8479',
  'container_title': 'Journal of High Energy Physics'},
 'JINST': {'ISSN': '1748-0221',
  'container_title': 'Journal of Instrumentation'},
 'Mod.Phys.Lett.': {'ISSN': '0217-7323',
  'container_title': 'Modern Physics Letters A'},
 'Nucl.Phys.': {'ISSN': '0550-3213', 'container_title': 'Nuclear Physics B'},
 'Phys.Lett.': {'ISSN': '0370-2693', 'container_title': 'Physics Letters B'},
 'Phys.Rept.': {'ISSN': '0370-1573', 'container_title': 'Physics Reports'

Normalize data

In [77]:
dfp=normalize(dfp)

Add Journal metadata

In [78]:
dfp['Year']=dfp['publication_info'].apply(
    lambda d: d.get('year') if isinstance(d,dict) else None).apply(
    lambda y: int(y) if isinstance(y,str) else 0)

dfp['Journal_name']=dfp['publication_info'].apply(lambda d: journal.get( 
                                        d.get('title') ).get('container_title') 
                                        if isinstance(d,dict) 
                                        else None)
dfp['ISSN']=dfp['publication_info'].apply(lambda d: journal.get( 
                                        d.get('title') ).get('ISSN') 
                                        if isinstance(d,dict) 
                                        else None)

Extract author metadata for affiliation UdeA

In [79]:
dfp['udea_authors']=dfp['authors'].apply(lambda l: [ #loop for each author in the list
        '{} {}'.format(d.get('first_name'), d.get('last_name')) #Extract full name 
         for d in l 
           if str(d.get('affiliation')).find('Antioquia U.')>-1 #For UdeA authors
                               ])
dfp['first_udea_author']=dfp['udea_authors'].str[0]

Prepare "Sostenibilidad" output

In [86]:
dfp.to_json('gfif.json.gz',compression='gzip')

In [87]:
kk=pd.read_json('gfif.json.gz',compression='gzip')

In [ ]:
sost=dfp[['title_TeX','Journal_name','first_udea_author','Year','ISSN','DOI']]

In [ ]:
sost[sost.Year>2017]#.to_excel('sost.xlsx',index=False)

In [81]:
%%bash
git commit -am 'lc'
git push origin master

[master ba8b47f] lc
 1 file changed, 348 insertions(+), 6 deletions(-)


To github.com:restrepo/inspire.git
   0448bba..ba8b47f  master -> master
